<a href="https://colab.research.google.com/github/SumitKaware/InsureAI-RAG-Based-ChatBot/blob/main/InsureAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gradio google-generativeai pandas numpy tensorflow langchain langchain-community chromadb langchain-huggingface langchain-chroma langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 9.9 MB/s eta 

In [20]:
import os
import glob
import gradio as gr
import zipfile
import shutil
from random import randint

In [2]:
#pip show gradio google-generativeai pandas numpy tensorflow langchain langchain-community chromadb langchain-huggingface langchain-chroma langchain-google-genai

In [6]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from google.colab import userdata
import google.generativeai as genai
from google.colab import files

In [7]:
api_key = userdata.get('GOOGLE_API_KEY')
if api_key:
    print("API Key looks good.")
else:
    print("There might be some problem with your API Key. Please check.")

MODEL = "gemini-2.0-flash-lite"
db_name = "InsureAI_VecVB"

API Key looks good.


In [9]:
genai.configure(api_key=api_key)
GEMINI = genai.GenerativeModel(MODEL)
prompt = "Explain how AI works in a few words"
response = GEMINI.generate_content(prompt)
response.text

'AI learns from data to solve problems.\n'

In [18]:
# def handle_upload(uploaded_files):
#     result_msg = ""

#     if not uploaded_files:
#         return "No files uploaded."

#     if len(uploaded_files) == 1 and uploaded_files[0].name.endswith('.zip'):
#         extract_path = '/content/uploaded_folder'
#         os.makedirs(extract_path, exist_ok=True)
#         with zipfile.ZipFile(uploaded_files[0].name, 'r') as zip_ref:
#             zip_ref.extractall(extract_path)
#         result_msg = f"[ZIP] Extracted to: {extract_path}"
#         print(result_msg)
#     elif len(uploaded_files) > 1:
#         extract_path = '/content/uploaded_folder'
#         os.makedirs(extract_path, exist_ok=True)
#         for file in uploaded_files:
#             shutil.move(file.name, os.path.join(extract_path, os.path.basename(file.name)))
#         result_msg = f"[MULTIPLE FILES] Uploaded files moved to: {extract_path}"
#         print(result_msg)
#     else:
#         extract_path = '/content/uploaded_folder'
#         os.makedirs(extract_path, exist_ok=True)
#         file = uploaded_files[0]
#         shutil.move(file.name, os.path.join(extract_path, os.path.basename(file.name)))
#         result_msg = f"[SINGLE FILE] Uploaded file moved to: {extract_path}"
#         print(result_msg)
#     return extract_path

# Gradio Interface
# try:
#   gr.Interface(
#     fn=handle_upload,
#     inputs=gr.File(file_types=[".zip", ".txt", ".csv", ".pdf", ".docx", ".png", ".jpg", "*"], file_count="multiple", label="Upload File(s) or ZIP Folder"),
#     outputs="text",
#     title="Flexible File/Folder Upload in Colab",
#     description="Upload a folder as .zip or multiple/single files. Automatically extracts or stores files in Colab."
# ).launch()
# except Exception as e:
#   print(f"Error: {e}")


In [51]:
def process_files(uploaded_files):
    # os.makedirs(folder, exist_ok=True)

    # processed_folders = []
    # target_directory = glob.glob("/content/drive/MyDrive/Colab-Notebooks/knowledge-base1/")
    # for folder_path in folders:
    #   folder_name = os.path.basename(folder_path)
    #   destination_path = os.path.join(target_directory, folder_name)

    # # Copy the entire folder to the target directory
    #   shutil.copytree(folder_path, destination_path)

    #   processed_folders.append(folder_name)

    # os.makedirs(folder, exist_ok=True)

    # processed_files = []
    # for file in files:
    #     file_path = os.path.join(folder, os.path.basename(file))  # Get filename
    #     shutil.copy(file, file_path)
    #     processed_files.append(os.path.basename(file))

    result_msg = ""

    if not uploaded_files:
        return "No files uploaded."

    if len(uploaded_files) == 1 and uploaded_files[0].name.endswith('.zip'):
        extract_path = '/content/uploaded_folder'
        os.makedirs(extract_path, exist_ok=True)
        with zipfile.ZipFile(uploaded_files[0].name, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        result_msg = f"[ZIP] Extracted to: {extract_path}"

        print(result_msg)
    elif len(uploaded_files) > 1:
        extract_path = '/content/uploaded_folder'
        os.makedirs(extract_path, exist_ok=True)
        for file in uploaded_files:
            shutil.move(file.name, os.path.join(extract_path, os.path.basename(file.name)))
        result_msg = f"[MULTIPLE FILES] Uploaded files moved to: {extract_path}"

        print(result_msg)
    else:
        extract_path = '/content/uploaded_folder'
        os.makedirs(extract_path, exist_ok=True)
        file = uploaded_files[0]
        shutil.move(file.name, os.path.join(extract_path, os.path.basename(file.name)))
        result_msg = f"[SINGLE FILE] Uploaded file moved to: {extract_path}"

        print(result_msg)

    processed_files = []
    for uploaded_file in uploaded_files:
        file_name = os.path.basename(uploaded_file.name)
        file_path = os.path.join(extract_path, file_name)
        shutil.copy(uploaded_file.name, file_path)
        processed_files.append(file_name)

    # Load documents using LangChain's DirectoryLoader
    text_loader_kwargs = {'autodetect_encoding': True}
    loader = DirectoryLoader(extract_path, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()

    # Assign filenames as metadata
    for doc in folder_docs:
        filename_md = os.path.basename(doc.metadata["source"])
        filename, _ = os.path.splitext(filename_md)
        doc.metadata["filename"] = filename

    documents = folder_docs

    # Split documents into chunks
    text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)

    # Initialize embeddings
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    # Delete previous vectorstore
    if os.path.exists(db_name):
        Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

    # Store in ChromaDB
    vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)

    # Retrieve results
    collection = vectorstore._collection
    result = collection.get(include=['embeddings', 'documents', 'metadatas'])

    llm = ChatGoogleGenerativeAI(temperature=0.7, model_name=MODEL)
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 35})
    global conversation_chain
    conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

    processed_text = "**Processed Files:**\n\n" + "\n".join(f"- {file}" for file in processed_files)
    return result, processed_text

In [52]:
def random_color():
        return f"rgb({randint(0,255)},{randint(0,255)},{randint(0,255)})"

In [53]:
def show_embeddings_2d(result):
    vectors = np.array(result['embeddings'])
    documents = result['documents']
    metadatas = result['metadatas']
    filenames = [metadata['filename'] for metadata in metadatas]
    filenames_unique = sorted(set(filenames))

    # color assignment
    color_map = {name: random_color() for name in filenames_unique}
    colors = [color_map[name] for name in filenames]

    tsne = TSNE(n_components=2, random_state=42,perplexity=4)
    reduced_vectors = tsne.fit_transform(vectors)

    # Create the 2D scatter plot
    fig = go.Figure(data=[go.Scatter(
        x=reduced_vectors[:, 0],
        y=reduced_vectors[:, 1],
        mode='markers',
        marker=dict(size=5,color=colors, opacity=0.8),
        text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(filenames, documents)],
        hoverinfo='text'
    )])

    fig.update_layout(
        title='2D Chroma Vector Store Visualization',
        scene=dict(xaxis_title='x',yaxis_title='y'),
        width=800,
        height=600,
        margin=dict(r=20, b=10, l=10, t=40)
    )

    return fig


In [54]:
def show_embeddings_3d(result):
    vectors = np.array(result['embeddings'])
    documents = result['documents']
    metadatas = result['metadatas']
    filenames = [metadata['filename'] for metadata in metadatas]
    filenames_unique = sorted(set(filenames))

    # color assignment
    color_map = {name: random_color() for name in filenames_unique}
    colors = [color_map[name] for name in filenames]

    tsne = TSNE(n_components=3, random_state=42)
    reduced_vectors = tsne.fit_transform(vectors)

    fig = go.Figure(data=[go.Scatter3d(
        x=reduced_vectors[:, 0],
        y=reduced_vectors[:, 1],
        z=reduced_vectors[:, 2],
        mode='markers',
        marker=dict(size=5, color=colors, opacity=0.8),
        text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(filenames, documents)],
        hoverinfo='text'
    )])

    fig.update_layout(
        title='3D Chroma Vector Store Visualization',
        scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
        width=900,
        height=700,
        margin=dict(r=20, b=10, l=10, t=40)
    )

    return fig

In [55]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

def visualise_data(result):
    fig_2d = show_embeddings_2d(result)
    fig_3d = show_embeddings_3d(result)
    return fig_2d,fig_3d

In [56]:
css = """
.btn {background-color: #1d53d1;}
"""

In [57]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("# Markdown-Based Q&A with Visualization")
    with gr.Row():
        file_input=gr.File(file_types=[".zip", ".txt", ".csv", ".pdf", ".docx", ".png", ".jpg", "*"], file_count="multiple", label="Upload File(s) or ZIP Folder")
        with gr.Column(scale=1):
            processed_output = gr.Markdown("Progress")
    with gr.Row():
        process_btn = gr.Button("Process Files",elem_classes=["btn"])
    with gr.Row():
        question = gr.Textbox(label="Chat ", lines=10)
        answer = gr.Markdown(label= "Response")
    with gr.Row():
        question_btn = gr.Button("Ask a Question",elem_classes=["btn"])
        clear_btn = gr.Button("Clear Output",elem_classes=["btn"])
    with gr.Row():
        plot_2d = gr.Plot(label="2D Visualization")
        plot_3d = gr.Plot(label="3D Visualization")
    with gr.Row():
        visualise_btn = gr.Button("Visualise Data",elem_classes=["btn"])

    result = gr.State([])
    # Action: When button is clicked, process files and update visualization
    clear_btn.click(fn=lambda:("", ""), inputs=[],outputs=[question, answer])
    process_btn.click(process_files, inputs=[file_input], outputs=[result,processed_output])
    question_btn.click(chat, inputs=[question], outputs= [answer])
    visualise_btn.click(visualise_data, inputs=[result], outputs=[plot_2d,plot_3d])

# Launch Gradio app
ui.launch(inbrowser=True)

/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1018: UserWarning: Expected 2 arguments for function <function chat at 0x7980b03c4b80>, received 1.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1022: UserWarning: Expected at least 2 arguments for function <function chat at 0x7980b03c4b80>, received 1.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e652a1e7a9540d4358.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [58]:
import os
import shutil
import zipfile
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import Chroma
# import pdb
# pdb.set_trace()

# Assuming db_name and MODEL are defined elsewhere in your code

def process_files(uploaded_files):
    global conversation_chain
    result_msg = ""
    db_name = '/content/knowledge-base'
    extract_path = '/content/uploaded_folder'
    os.makedirs(extract_path, exist_ok=True)
    processed_files = []

    if not uploaded_files:
        return "No files uploaded."

    if len(uploaded_files) == 1 and uploaded_files[0].endswith('.zip'):
        try:
            with zipfile.ZipFile(uploaded_files[0], 'r') as zip_ref:
                zip_ref.extractall(extract_path)
            result_msg = f"[ZIP] Extracted to: {extract_path}"
            print(result_msg)
            processed_files.append(os.path.basename(uploaded_files[0]))
        except zipfile.BadZipFile:
            return "Error: Invalid ZIP file."
        except Exception as e:
            return f"Error extracting ZIP: {e}"
    elif len(uploaded_files) > 0:
        for file_path in uploaded_files:
            try:
                shutil.move(file_path, os.path.join(extract_path, os.path.basename(file_path)))
                processed_files.append(os.path.basename(file_path))
            except Exception as e:
                return f"Error moving file '{os.path.basename(file_path)}': {e}"
        result_msg = f"[MULTIPLE FILES] Uploaded files moved to: {extract_path}"
        print(result_msg)
    else:
        return "No files were processed."

    documents = []
    try:
        # Load documents using LangChain's DirectoryLoader
        text_loader_kwargs = {'autodetect_encoding': True}
        loader = DirectoryLoader(extract_path, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
        folder_docs = loader.load()
        documents.extend(folder_docs)
    except Exception as e:
        return f"Error loading documents: {e}"

    try:
        # Assign filenames as metadata
        for doc in documents:
            if "source" in doc.metadata:
                filename_md = os.path.basename(doc.metadata["source"])
                filename, _ = os.path.splitext(filename_md)
                doc.metadata["filename"] = filename
            else:
                doc.metadata["filename"] = "unknown" # Handle cases without 'source'
    except Exception as e:
        return f"Error processing document metadata: {e}"

    chunks = []
    try:
        # Split documents into chunks
        text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=200)
        chunks = text_splitter.split_documents(documents)
    except Exception as e:
        return f"Error splitting documents: {e}"

    vectorstore = None
    try:
        # Initialize embeddings
        embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

        # Delete previous vectorstore
        if os.path.exists(db_name):
            Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

        # Store in ChromaDB
        vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
    except Exception as e:
        return f"Error creating or accessing vectorstore: {e}"

    result = None
    try:
        # Retrieve results
        if vectorstore:
            collection = vectorstore._collection
            result = collection.get(include=['embeddings', 'documents', 'metadatas'])
    except Exception as e:
        return f"Error retrieving results from vectorstore: {e}"

    conversation_chain = None
    try:
        # Initialize LLM and conversation chain
        llm = ChatGoogleGenerativeAI(temperature=0.7, model_name=MODEL)
        memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
        retriever = vectorstore.as_retriever(search_kwargs={"k": 35})
        conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)
    except Exception as e:
        return f"Error initializing LLM or conversation chain: {e}"

    processed_text = "**Processed Files:**\n\n" + "\n".join(f"- {os.path.basename(f)}" for f in processed_files)
    return result, processed_text

In [61]:
from google.colab import auth
from google.cloud import storage  # Example: importing a Cloud Storage client

try:
    auth.authenticate_user()
    print("Successfully authenticated with Google Colab user credentials.")
except Exception as e:
    print(f"Authentication error: {e}")

# Now you can use Google Cloud client libraries
#client = storage.Client()
# ... your Google Cloud code ...

Successfully authenticated with Google Colab user credentials.


In [ ]:
css = """
.btn {background-color: #1d53d1;}
"""

with gr.Blocks(css=css) as ui:
    gr.Markdown("# Markdown-Based Q&A with Visualization")
    with gr.Row():
        file_input=gr.File(file_types=[".zip", ".txt", ".csv", ".pdf", ".docx", ".png", ".jpg", "*"], file_count="multiple", label="Upload File(s) or ZIP Folder")
        with gr.Column(scale=1):
            processed_output = gr.Markdown("Progress")
    with gr.Row():
        process_btn = gr.Button("Process Files",elem_classes=["btn"])
    with gr.Row():
        question = gr.Textbox(label="Chat ", lines=10)
        answer = gr.Markdown(label= "Response")
    with gr.Row():
        question_btn = gr.Button("Ask a Question",elem_classes=["btn"])
        clear_btn = gr.Button("Clear Output",elem_classes=["btn"])
    with gr.Row():
        plot_2d = gr.Plot(label="2D Visualization")
        plot_3d = gr.Plot(label="3D Visualization")
    with gr.Row():
        visualise_btn = gr.Button("Visualise Data",elem_classes=["btn"])

    result = gr.State([])
    # Action: When button is clicked, process files and update visualization
    clear_btn.click(fn=lambda:("", ""), inputs=[],outputs=[question, answer])
    process_btn.click(process_files, inputs=[file_input], outputs=[result,processed_output])
    # question_btn.click(chat, inputs=[question], outputs= [answer])
    # visualise_btn.click(visualise_data, inputs=[result], outputs=[plot_2d,plot_3d])

# Launch Gradio app
ui.launch(inbrowser=True, debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://526476a6d2924b09ab.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[ZIP] Extracted to: /content/uploaded_folder


<ipython-input-58-1bd4a2a5b4e0>:86: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2156, in process_api
    data = a